In [1]:
import pandas as pd, numpy as np, os
from get_draws.api import get_draws
import matplotlib.pyplot as plt
import matplotlib.backends.backend_pdf
from matplotlib.backends.backend_pdf import PdfPages

In [2]:
output_dir ='/ihme/costeffectiveness/results/vivarium_ciff_sam/v6.0.1_vicious_cycle/ciff_sam/2022_02_23_17_26_53/count_data/'

In [3]:
figname = 'model_6.0.1'

In [4]:
os.listdir(output_dir)

['wasting_transition_count.hdf',
 'wasting_transition_count.csv',
 'wasting_state_person_time.hdf',
 'deaths.csv',
 'wasting_state_person_time.csv',
 'deaths.hdf',
 'disease_transition_count.csv',
 'births.csv',
 'population.csv',
 'stunting_state_person_time.hdf',
 'ylds.csv',
 'ylls.csv',
 'disease_state_person_time.csv',
 'population.hdf',
 'ylls.hdf',
 'ylds.hdf',
 'disease_state_person_time.hdf',
 'births.hdf',
 'disease_transition_count.hdf',
 'stunting_state_person_time.csv']

In [5]:
wasting_pt = pd.read_csv(output_dir +'wasting_state_person_time.csv')
wasting_pt.head()

,Unnamed: 0,sex,year,cause,measure,input_draw,scenario,value,diarrhea,x_factor,sq_lns,mam_treatment,sam_treatment,age
0,0,female,2022,mild_child_wasting,state_person_time,29,baseline,0.0,cat2,cat2,covered,covered,covered,early_neonatal
1,1,female,2022,mild_child_wasting,state_person_time,29,baseline,0.0,cat1,cat2,covered,covered,covered,early_neonatal
2,2,female,2022,mild_child_wasting,state_person_time,29,baseline,0.0,cat2,cat1,covered,covered,covered,early_neonatal
3,3,female,2022,mild_child_wasting,state_person_time,29,baseline,0.0,cat1,cat1,covered,covered,covered,early_neonatal
4,4,female,2022,mild_child_wasting,state_person_time,29,baseline,0.0,cat2,cat2,covered,uncovered,covered,early_neonatal


In [48]:
def calculate_child_growth_exposure_prevalence(risk, groupby_cols):
    state_pt = pd.read_csv(output_dir +f'{risk}_state_person_time.csv')
    #state_pt = state_pt.loc[state_pt.scenario=='baseline']
    state_pt = state_pt.groupby(groupby_cols).sum()[['value']].reset_index()
    if risk=='wasting':
        state_pt_exposed = state_pt.loc[state_pt.cause.str.contains('acute_malnutrition')]
        state_pt_exposed['cause'] = 'wasted'
    else:
        state_pt_exposed = state_pt.loc[state_pt.cause.isin(['cat1','cat2'])]
        state_pt_exposed['cause'] = 'stunted'
    state_pt_with_exposed = (pd.concat([state_pt, 
                          state_pt_exposed.groupby(groupby_cols).sum().reset_index()],
                         ignore_index=True)
                .set_index(groupby_cols))
    prev = state_pt_with_exposed / state_pt.groupby([c for c in groupby_cols if c != 'cause']).sum()
    #prev = prev.groupby([c for c in groupby_cols if c != 'input_draw']).describe(percentiles=[0.025,0.975]).sort_index()
    return prev

In [63]:
w = calculate_child_growth_exposure_prevalence('wasting', ['scenario','input_draw','sex','age','cause','diarrhea']).reset_index()
pr = (w.loc[w.diarrhea=='cat1'].set_index(['scenario','input_draw','sex','age','cause'])[['value']]
     / w.loc[w.diarrhea=='cat2'].set_index(['scenario','input_draw','sex','age','cause'])[['value']]).reset_index()
pr = pr.loc[pr.age.isin(['6-11_months','12_to_23_months','2_to_4'])].groupby(['scenario','sex','age','cause']).describe(percentiles=[0.025,0.975])
pr = pr[['value']]
pr
# these are a bit more dramatic than I would expect...
# could this be due to a correlation between stunting and wasting??

value  \
                                                                      count   
scenario          sex    age             cause                                
baseline          female 12_to_23_months mild_child_wasting            12.0   
                                         moderate_acute_malnutrition   12.0   
                                         severe_acute_malnutrition     12.0   
                                         susceptible_to_child_wasting  12.0   
                                         wasted                        12.0   
...                                                                     ...   
wasting_treatment male   6-11_months     mild_child_wasting            12.0   
                                         moderate_acute_malnutrition   12.0   
                                         severe_acute_malnutrition     12.0   
                                         susceptible_to_child_wasting  12.0   
                                         wasted                        12.0   

                                                                                 \
                                                                           mean   
scenario          sex    age             cause                                    
baseline          female 12_to_23_months mild_child_wasting            1.073695   
                                         moderate_acute_malnutrition   1.753944   
                                         severe_acute_malnutrition     1.167733   
                                         susceptible_to_child_wasting  0.902966   
                                         wasted                        1.638201   
...                                                                         ...   
wasting_treatment male   6-11_months     mild_child_wasting            1.079806   
                                         moderate_acute_malnutrition   1.652923   
                                         severe_acute_malnutrition     1.161017   
                                         susceptible_to_child_wasting  0.878732   
                                         wasted                        1.516027   

                                                                                 \
                                                                            std   
scenario          sex    age             cause                                    
baseline          female 12_to_23_months mild_child_wasting            0.007279   
                                         moderate_acute_malnutrition   0.032298   
                                         severe_acute_malnutrition     0.035692   
                                         susceptible_to_child_wasting  0.004541   
                                         wasted                        0.028263   
...                                                                         ...   
wasting_treatment male   6-11_months     mild_child_wasting            0.007539   
                                         moderate_acute_malnutrition   0.039214   
                                         severe_acute_malnutrition     0.030061   
                                         susceptible_to_child_wasting  0.003991   
                                         wasted                        0.031510   

                                                                                 \
                                                                            min   
scenario          sex    age             cause                                    
baseline          female 12_to_23_months mild_child_wasting            1.059937   
                                         moderate_acute_malnutrition   1.693350   
                                         severe_acute_malnutrition     1.115873   
                                         susceptible_to_child_wasting  0.894195   
                                         wasted         

In [64]:
pr.reset_index().loc[(pr.reset_index().scenario=='baseline')
                    &(pr.reset_index().age=='2_to_4')]

scenario     sex     age                         cause value            \
                                                           count      mean   
5   baseline  female  2_to_4            mild_child_wasting  12.0  1.077452   
6   baseline  female  2_to_4   moderate_acute_malnutrition  12.0  1.814925   
7   baseline  female  2_to_4     severe_acute_malnutrition  12.0  1.197916   
8   baseline  female  2_to_4  susceptible_to_child_wasting  12.0  0.914281   
9   baseline  female  2_to_4                        wasted  12.0  1.728968   
20  baseline    male  2_to_4            mild_child_wasting  12.0  1.074943   
21  baseline    male  2_to_4   moderate_acute_malnutrition  12.0  1.757672   
22  baseline    male  2_to_4     severe_acute_malnutrition  12.0  1.198437   
23  baseline    male  2_to_4  susceptible_to_child_wasting  12.0  0.904286   
24  baseline    male  2_to_4                        wasted  12.0  1.658332   

                                                                
         std       min      2.5%       50%     97.5%       max  
5   0.005639  1.069024  1.069542  1.076839  1.087850  1.088897  
6   0.021368  1.782972  1.784600  1.815050  1.852456  1.857195  
7   0.028952  1.146822  1.155676  1.193328  1.251797  1.253103  
8   0.002693  0.909284  0.910154  0.913719  0.918712  0.919108  
9   0.021848  1.703415  1.704340  1.722370  1.767755  1.769754  
20  0.003917  1.067755  1.068800  1.074598  1.081810  1.082658  
21  0.023466  1.716231  1.716596  1.760151  1.790470  1.792201  
22  0.030132  1.144921  1.150260  1.199744  1.234567  1.235137  
23  0.003407  0.897568  0.898516  0.904289  0.909909  0.910372  
24  0.022924  1.615460  1.616074  1.660974  1.687724  1.687875

In [61]:
f = (w.groupby(['scenario','input_draw','age','diarrhea','cause']).sum() 
 / w.groupby(['scenario','input_draw','age','diarrhea']).sum()).groupby(['scenario','age','cause','diarrhea']).mean().reset_index()
f.loc[(f.age.isin(['2_to_4']))
     &(f.scenario=='baseline')]

,scenario,age,cause,diarrhea,value
20,baseline,2_to_4,mild_child_wasting,cat1,0.194450
21,baseline,2_to_4,mild_child_wasting,cat2,0.189145
22,baseline,2_to_4,moderate_acute_malnutrition,cat1,0.097004
23,baseline,2_to_4,moderate_acute_malnutrition,cat2,0.056920
24,baseline,2_to_4,severe_acute_malnutrition,cat1,0.012471
25,baseline,2_to_4,severe_acute_malnutrition,cat2,0.010893
26,baseline,2_to_4,susceptible_to_child_wasting,cat1,0.586600
27,baseline,2_to_4,susceptible_to_child_wasting,cat2,0.675230
28,baseline,2_to_4,wasted,cat1,0.109475
29,baseline,2_to_4,wasted,cat2,0.067812


In [62]:
1247/1089

1.145087235996327

In [55]:
# prevalence ratios are exaggerated in the sqlns scenario, which impacts both stunting and wasting

pr.reset_index().loc[(pr.reset_index().age=='12_to_23_months')
                    &(pr.reset_index().scenario.isin(['baseline','sqlns']))].set_index(['sex','cause','scenario'])[['value']].sort_index()

value                      \
                                             count      mean       std   
sex    cause                        scenario                             
female mild_child_wasting           baseline  12.0  1.073695  0.007279   
                                    sqlns     12.0  1.087731  0.007386   
       moderate_acute_malnutrition  baseline  12.0  1.753944  0.032298   
                                    sqlns     12.0  1.827142  0.032447   
       severe_acute_malnutrition    baseline  12.0  1.167733  0.035692   
                                    sqlns     12.0  1.189464  0.046835   
       susceptible_to_child_wasting baseline  12.0  0.902966  0.004541   
                                    sqlns     12.0  0.904129  0.004375   
       wasted                       baseline  12.0  1.638201  0.028263   
                                    sqlns     12.0  1.710019  0.027218   
male   mild_child_wasting           baseline  12.0  1.076553  0.007715   
                                    sqlns     12.0  1.091881  0.007797   
       moderate_acute_malnutrition  baseline  12.0  1.696374  0.025834   
                                    sqlns     12.0  1.768381  0.025891   
       severe_acute_malnutrition    baseline  12.0  1.174148  0.042613   
                                    sqlns     12.0  1.204300  0.054079   
       susceptible_to_child_wasting baseline  12.0  0.878225  0.005315   
                                    sqlns     12.0  0.879029  0.004915   
       wasted                       baseline  12.0  1.570173  0.022891   
                                    sqlns     12.0  1.640873  0.027702   

                                                                            \
                                                   min      2.5%       50%   
sex    cause                        scenario                                 
female mild_child_wasting           baseline  1.059937  1.061262  1.075347   
                                    sqlns     1.078320  1.078400  1.088911   
       moderate_acute_malnutrition  baseline  1.693350  1.700078  1.756519   
                                    sqlns     1.783604  1.784326  1.828673   
       severe_acute_malnutrition    baseline  1.115873  1.117812  1.159939   
                                    sqlns     1.124839  1.131598  1.186584   
       susceptible_to_child_wasting baseline  0.894195  0.895692  0.902383   
                                    sqlns     0.895833  0.897156  0.903167   
       wasted                       baseline  1.578320  1.585355  1.639819   
                                    sqlns     1.667558  1.670022  1.713934   
male   mild_child_wasting           baseline  1.057368  1.061043  1.078715   
                                    sqlns     1.077070  1.078831  1.091153   
       moderate_acute_malnutrition  baseline  1.645379  1.648096  1.700970   
                                    sqlns     1.719903  1.722608  1.769651   
       severe_acute_malnutrition    baseline  1.103540  1.108204  1.178067   
                                    sqlns     1.119527  1.127959  1.197616   
       susceptible_to_child_wasting baseline  0.870543  0.871219  0.877163   
                                    sqlns     0.870732  0.871796  0.878571   
       wasted                       baseline  1.534427  1.534438  1.568120   
                                    sqlns     1.600610  1.600776  1.636647   

                                                                  
                                                 97.5%       max  
sex    cause                        scenario                      
female mild_child_wasting           baseline  1.083720  1.084806  
                                    sqlns     1.099882  1.100798  
       moderate_acute_malnutrition  baseline  1.802725  1.807572  
                                    sqlns     1.882753  1.892737  
       severe_acute_malnutrition    baseline  1.226135  1.227971  
           

In [8]:
def compute_wasting_transition_rate(transition_count_parameter, person_time_state):
    rate = (((wasting_transitions.loc[wasting_transitions.measure==f'{transition_count_parameter}']
           .set_index(['input_draw','age']).drop(columns='measure'))
          / (wasting_person_time.loc[wasting_person_time.cause==f'{person_time_state}']
             .set_index(['input_draw','age']).drop(columns='cause')))
          .groupby(['age']).describe(percentiles=[0.025,0.975]).reset_index())
    #rate = rate.loc[rate.age.isin(ages)]
    #rate['age_start'] = np.where(rate.age=='6-11_months',0.5,
    #                            np.where(rate.age=='12_to_23_months', 1.0, 2.0))
    #rate['age_end'] = np.where(rate.age=='6-11_months',1.0,
    #                            np.where(rate.age=='12_to_23_months', 2.0, 5.0))
    rate['source_state'] = person_time_state.replace('_event_count','')
    rate['sink_state'] = transition_count_parameter
    return rate#.set_index(['sex','age'])[['value']]

In [9]:
wasting_transitions = pd.read_csv(output_dir +'wasting_transition_count.csv')
wasting_person_time = pd.read_csv(output_dir +'wasting_state_person_time.csv')

In [10]:
under_six_mo = wasting_transitions.loc[wasting_transitions.age.isin(['early_neonatal',
                                                                    'late_neonatal',
                                                                    '1-5_months'])]
non_zero = under_six_mo.loc[under_six_mo['value']!=0]
non_zero

# cool, no more transitions under six months of age

,Unnamed: 0,sex,year,measure,input_draw,scenario,value,diarrhea,x_factor,sq_lns,mam_treatment,sam_treatment,age


In [11]:
non_zero.measure.unique()

array([], dtype=object)

In [41]:
transitions = ['mild_child_wasting_to_moderate_acute_malnutrition_event_count']
cause = 'mild_child_wasting'
transition_count = (wasting_transitions.loc[wasting_transitions.measure.isin(transitions)]
                    .groupby(['scenario','input_draw','age','sex','diarrhea']).sum()[['value']])
state_pt = (wasting_person_time.loc[wasting_person_time.cause==cause]
           .groupby(['scenario','input_draw','age','sex','diarrhea']).sum()[['value']])
rate = (transition_count / state_pt).reset_index()
rate = rate.loc[(rate.scenario=='baseline')&(rate.age.isin(['6-11_months','12_to_23_months','2_to_4']))]
rate_d = rate.loc[rate.diarrhea=='cat1'].set_index(['scenario','input_draw','age','sex'])[['value']]
rate_s = rate.loc[rate.diarrhea=='cat2'].set_index(['scenario','input_draw','age','sex'])[['value']]
rr = rate_d / rate_s
rr.groupby(['scenario','sex','age']).describe(percentiles=[0.025,0.975])


# 14-18 validation target for mild->MAM
# a little low

value                                  \
                                count       mean       std        min   
scenario sex    age                                                     
baseline female 12_to_23_months  12.0  14.553068  0.799848  13.484461   
                2_to_4           12.0  15.853482  0.828126  14.724823   
                6-11_months      12.0  15.881428  1.845991  12.651218   
         male   12_to_23_months  12.0  12.921521  0.836986  11.892603   
                2_to_4           12.0  14.756392  0.662500  13.661183   
                6-11_months      12.0  13.568490  1.494956  11.902298   

                                                                             
                                      2.5%        50%      97.5%        max  
scenario sex    age                                                          
baseline female 12_to_23_months  13.568858  14.333778  16.090081  16.229001  
                2_to_4           14.796187  15.766160  17.429078  17.615636  
                6-11_months      12.906163  16.283501  19.097712  19.614342  
         male   12_to_23_months  11.985251  12.818936  14.625797  15.048776  
                2_to_4           13.765733  14.811440  15.859090  15.951176  
                6-11_months      11.941394  13.241137  16.694782  17.041401

In [42]:
transitions = ['susceptible_to_child_wasting_to_mild_child_wasting_event_count']
cause = 'susceptible_to_child_wasting'
transition_count = (wasting_transitions.loc[wasting_transitions.measure.isin(transitions)]
                    .groupby(['scenario','input_draw','age','sex','diarrhea']).sum()[['value']])
state_pt = (wasting_person_time.loc[wasting_person_time.cause==cause]
           .groupby(['scenario','input_draw','age','sex','diarrhea']).sum()[['value']])
rate = (transition_count / state_pt).reset_index()
rate = rate.loc[(rate.scenario=='baseline')&(rate.age.isin(['6-11_months','12_to_23_months','2_to_4']))]
rate_d = rate.loc[rate.diarrhea=='cat1'].set_index(['scenario','input_draw','age','sex'])[['value']]
rate_s = rate.loc[rate.diarrhea=='cat2'].set_index(['scenario','input_draw','age','sex'])[['value']]
rr = rate_d / rate_s
rr.groupby(['scenario','sex','age']).describe(percentiles=[0.025,0.975])


# 3.8 - 4.2 validation target for TMREL->mild
# a little high.. 

value                                          \
                                count      mean       std       min      2.5%   
scenario sex    age                                                             
baseline female 12_to_23_months  12.0  3.979820  0.059403  3.884079  3.895462   
                2_to_4           12.0  3.804762  0.070312  3.701412  3.705326   
                6-11_months      12.0  4.517451  0.149684  4.250942  4.285410   
         male   12_to_23_months  12.0  4.285908  0.056902  4.200484  4.203051   
                2_to_4           12.0  3.958867  0.059118  3.890618  3.891612   
                6-11_months      12.0  4.669684  0.140739  4.474051  4.476728   

                                                               
                                      50%     97.5%       max  
scenario sex    age                                            
baseline female 12_to_23_months  3.968667  4.082557  4.098631  
                2_to_4           3.805075  3.925421  3.945452  
                6-11_months      4.517612  4.750955  4.782175  
         male   12_to_23_months  4.281743  4.392314  4.403767  
                2_to_4           3.958314  4.059184  4.063395  
                6-11_months      4.665642  4.884611  4.887749

In [44]:
transitions = ['moderate_acute_malnutrition_to_severe_acute_malnutrition_event_count']
cause = 'moderate_acute_malnutrition'
transition_count = (wasting_transitions.loc[wasting_transitions.measure.isin(transitions)]
                    .groupby(['scenario','input_draw','age','sex','diarrhea']).sum()[['value']])
state_pt = (wasting_person_time.loc[wasting_person_time.cause==cause]
           .groupby(['scenario','input_draw','age','sex','diarrhea']).sum()[['value']])
rate = (transition_count / state_pt).reset_index()
rate = rate.loc[(rate.scenario=='baseline')&(rate.age.isin(['6-11_months','12_to_23_months','2_to_4']))]
rate_d = rate.loc[rate.diarrhea=='cat1'].set_index(['scenario','input_draw','age','sex'])[['value']]
rate_s = rate.loc[rate.diarrhea=='cat2'].set_index(['scenario','input_draw','age','sex'])[['value']]
rr = rate_d / rate_s
rr.groupby(['scenario','age','sex']).describe(percentiles=[0.025,0.975])

# validation target ~3 for MAM->SAM
# looks a little high...

value                                          \
                                count      mean       std       min      2.5%   
scenario age             sex                                                    
baseline 12_to_23_months female  12.0  3.598032  0.307464  3.170689  3.170702   
                         male    12.0  3.576622  0.298912  3.181319  3.223407   
         2_to_4          female  12.0  3.623991  0.278682  3.210827  3.224566   
                         male    12.0  3.656198  0.271786  3.335468  3.338667   
         6-11_months     female  12.0  3.849539  0.470348  3.286650  3.303534   
                         male    12.0  3.851708  0.386301  3.213791  3.264263   

                                                               
                                      50%     97.5%       max  
scenario age             sex                                   
baseline 12_to_23_months female  3.559829  4.087840  4.114571  
                         male    3.523517  4.120446  4.150242  
         2_to_4          female  3.659106  4.068126  4.139696  
                         male    3.592506  4.152377  4.188534  
         6-11_months     female  3.724478  4.705128  4.755001  
                         male    3.891164  4.424369  4.490524